In [ ]:
%pylab inline
rcParams['figure.figsize'] = (10, 4) #wide graphs by default
from __future__ import print_function
from __future__ import division
from IPython.display import Audio

Convolution:

$$(f * g )(\tau) \stackrel{\mathrm{def}}{=}\ \int_{-\infty}^\infty f(t)\, g(\tau - t)\, d\tau$$

Doesn't this look a lot like cross-correlation?

$$(f \star g)(t)\ \stackrel{\mathrm{def}}{=} \int_{-\infty}^{\infty} f^*(\tau)\ g(t+\tau)\,d\tau$$

Discrete Convolution:

$$(f * g)[n]\ \stackrel{\mathrm{def}}{=}\ \sum_{m=-\infty}^\infty f[m]\, g[n - m]$$

Circular discrete convolution:

$$(f * g_N)[n] = \sum_{m=0}^{N-1} f[m]\ g_N[n-m]$$

https://en.wikipedia.org/wiki/Convolution

![http://upload.wikimedia.org/wikipedia/commons/6/6a/Convolution_of_box_signal_with_itself2.gif](http://upload.wikimedia.org/wikipedia/commons/6/6a/Convolution_of_box_signal_with_itself2.gif)

![http://upload.wikimedia.org/wikipedia/commons/b/b9/Convolution_of_spiky_function_with_box2.gif](http://upload.wikimedia.org/wikipedia/commons/b/b9/Convolution_of_spiky_function_with_box2.gif)


Discrete Convolution can be understood as placing a scaled copy of the [Impulse Response](https://en.wikipedia.org/wiki/Impulse_response) (usually the shorter of the source signals) at every sample of the other. In other words, we treat each sample in the input signal as an impulse that "activates" the response.

### Impulse Response
> In signal processing, the impulse response, or impulse response function (IRF), of a dynamic system is its output when presented with a brief input signal, called an impulse. More generally, an impulse response is the reaction of any dynamic system in response to some external change. In both cases, the impulse response describes the reaction of the system as a function of time

(Check out the [impulse response database](http://www.openairlib.net/auralizationdb))

In [ ]:
impulse = r_[1, zeros(15)]
stem(impulse)
ylim((-1.1, 1.1))
pass

Now make an impulse response:

In [ ]:
impulseResponse = [0.5, -1, 0, 0, 0.2, 0]
stem(impulseResponse)
ylim((-1.1, 1.1))
pass

Convolving an impulse with the response:

In [ ]:
stem(convolve(impulse, impulseResponse))
ylim((-1.1, 1.1))
pass

When we convolve an impulse signal with some "impulse response", we can see that it's like placing a scaled copy of the impulse response at the position of the impulse.

In [ ]:
impulse2 = r_[zeros(5), -0.5, zeros(10)]
stem(impulse2)
ylim((-1.1, 1.1))
pass

In [ ]:
stem(convolve(impulse2, impulseResponse))
ylim((-1.1, 1.1))
pass

Because the process is [linear](https://en.wikipedia.org/wiki/Linear_system) and the [superposition principle](https://en.wikipedia.org/wiki/Superposition_principle
) applies (we can superimpose the result instead of superimposing the input with the same result):

In [ ]:
subplot(121)
added_after = convolve(impulse, impulseResponse) + convolve(impulse2, impulseResponse)
stem(added_after)

added_before = convolve(impulse + impulse2, impulseResponse)
subplot(122)
stem(added_before)

pass

Convolve these two signals manually (by hand):

In [ ]:
f = [   1,   2,   3,  0,  0,  0]
g = [-0.5, 0.5, 0.1]

print(len(f) + len(g) - 1)

subplot(121)
stem(arange(6), f)
xlim((-0.5, 6.5))

subplot(122)
stem(arange(3), g)
xlim((-0.5, 6.5))
pass

In [ ]:
subplot(121)
stem(array([-0.5, -0.5, -0.4, 1.7, 0.3]))

subplot(122)
stem(convolve(f, g))
pass

What's the difference between the two above? Shouldn't they be the same? (The convolved signal may be len(f) + len(g) - 1)

Let's try another impulse response..

In [ ]:
g = array([0.1, 0.2, -0.1])
stem(g)
pass

In [ ]:
fg0 = f[0] * g  # f[0] == 1
subplot(211)
stem(fg0)
subplot(212)
stem(g)
pass

In [ ]:
fg1 = f[1] * g # f[1] == 2
stem(fg1)
pass

But we need to shift it in time to place it at the position of f[1]

In [ ]:
fg1 = r_[0, fg1] # shift over by 1
stem(fg1)
pass

In [ ]:
fg2 = f[2] * g
fg2 = r_[0, 0, fg2] # shift over by 2
stem(fg2)
pass

Now we need to add them

In [ ]:
def hide_axes(ax):
    ax.spines["top"].set_visible(False)  
    ax.spines["right"].set_visible(False) 
    ax.spines["bottom"].set_visible(False) 
    ax.spines["left"].set_visible(False)
    tick_params(axis="both", which="both", bottom="off", top="off",  
                labelbottom="off", left="off", right="off", labelleft="on")
    

In [ ]:
subplot(511)
stem(fg0)
xlim(0, 6)
ylim(-0.33, 0.61)
hide_axes(gca())

subplot(512)
stem(fg1)
xlim(0, 6)
ylim(-0.33, 0.62)
hide_axes(gca())

subplot(513)
stem(fg2)
xlim(0, 6)
ylim(-0.33, 0.62)
hide_axes(gca())

subplot(514)
stem(f[0] * r_[g, zeros(4)] + f[1] * r_[0, g, zeros(3)] + f[2] * r_[0, 0, g, zeros(2)])
xlim(0, 6)
ylim(-0.33, 0.62)

subplot(515)
stem(convolve(f, g))
xlim(0, 6)
ylim(-0.33, 0.62)

print(convolve(f, g))

gcf().set_figheight(8)

In [ ]:
def apply_ir(f, n, g):
    fg = f[n] * g # here's where the scaling happens!
    # this is the shifting
    fg = r_[zeros(n), fg, zeros(len(g) + len(f) - len(fg) - n - 1 )]
    return fg

In [ ]:
f, g

In [ ]:
fg = apply_ir(f, 2, g)
stem(fg)
pass

In [ ]:
#
convolved = zeros(8)
for i in range(len(f)):
    convolved += apply_ir(f, i, g)

In [ ]:
stem(convolved)
pass

In [ ]:
stem(convolve(f, g))
pass

Same result!

### Filtering and Convolution

Convolution can be used the mechanism to realize [Finite impulse response (FIR)](https://en.wikipedia.org/wiki/Finite_impulse_response) filters.

In [ ]:
signal = random.random(4096) * 2 - 1
originalSpectrum = specgram(signal, Fs=44100)
pass

In [ ]:
impulseResponse = ones(30) / 30.0

In [ ]:
plot(signal)
figure()
stem(impulseResponse)
pass

In [ ]:
signalNew = convolve(signal, impulseResponse)

In [ ]:
subplot(211)
plot(signal)
xlim(0, 20)
subplot(212)
plot(signalNew)
xlim(0, 20)


What just happened? What did we just do?

In [ ]:
Audio(data = signal, rate = 44100)

In [ ]:
Audio(data = signalNew, rate = 44100)

In [ ]:
spectrum, frequency, time, image = specgram(signalNew, Fs=44100)
pass

In [ ]:
subplot(211)
semilogx(spectrum[:, 10])
subplot(212)
semilogx(originalSpectrum[0][:, 10])
pass

Each sample of the output ends up being an average of up to 30 of it's neighbors. This "rounds out" sharp edges in the signal. This is a low-pass filter.

Now let's look at how we might expect this filter to effect an arbitrary signal.

In [ ]:
from scipy.signal import freqz # grab freqz which tells you about the filter you made

normalizedFrequency, frequencyResponse = freqz(impulseResponse)

plot(normalizedFrequency, abs(frequencyResponse))
ylabel('magnitude')
twinx()
plot(normalizedFrequency, angle(frequencyResponse), 'r')
ylabel('phase')
pass

Convolution in the time domain is multiplication in the frequency domain! This is good because convolution is $O(n^2)$ and the FFT is $O(n\log(n))$, so that means we can do convolution in $O(n\log(n))$.

In [ ]:
plot(abs(fft.rfft(impulseResponse)), 'o-')
pass

In [ ]:
plot(abs(fft.rfft(impulseResponse, n=512))) # n= adds zero padding
twinx()
plot(angle(fft.rfft(impulseResponse, n=512)), 'r')
pass

What's happening with phase here?

In [ ]:
window = signalNew[:256]
semilogx(abs(rfft(window)))
pass

In [ ]:
semilogx(abs(rfft(window) * rfft(impulseResponse, n=256))) # equivalent to convolution
pass

Convolution can be used for filtering but also with long IRs for reverb.

By: Andrés Cabrera mantaraya36@gmail.com
For MAT course MAT 201A at UCSB

Adapted by Karl Yerkes

This ipython notebook is licensed under the CC-BY-NC-SA license: http://creativecommons.org/licenses/by-nc-sa/4.0/

![http://i.creativecommons.org/l/by-nc-sa/3.0/88x31.png](http://i.creativecommons.org/l/by-nc-sa/3.0/88x31.png)